In [1]:
"""
Random Policy Example for EldenGym

This notebook demonstrates:
1. Creating an environment with frame streaming
2. Using multi-binary action space with key toggles
3. Accessing dict observations (frame + memory attributes)
4. Applying preprocessing wrappers
5. Running a simple random policy
"""

import time
import eldengym

In [2]:
eldengym.list_envs()

['Margit-v0']

In [3]:
env = eldengym.make("Margit-v0", launch_game=False)

Skipping game launch (launch_game=False)
Initializing Siphon...
Loading config from: C:\Users\DM\Projects\eldengym\eldengym\files\Margit-v0\er_siphon_config.toml
Sending configuration to server...
Server response: Process configuration set successfully
Waiting 2 seconds for process to be ready...
Initializing memory subsystem...
Server response: Memory initialized successfully
Process ID: 40620
Initializing input subsystem...
Server response: Input initialized successfully
Initializing capture subsystem...
Server response: Capture initialized successfully
Window size: 3840x2160

=== Initialization Complete! ===
All subsystems initialized successfully.
Checking server status...
Server status: {'success': True, 'message': 'Server status retrieved successfully', 'config_set': True, 'memory_initialized': True, 'input_initialized': True, 'capture_initialized': True, 'process_name': 'eldenring.exe', 'window_name': 'ELDEN RING', 'process_id': 40620}
Starting frame stream...
Frame stream start

In [4]:
env.client.get_attribute("HeroLocalPosX")

{'success': True,
 'message': 'HeroLocalPosX read successfully',
 'value': 4.400001525878906,
 'value_type': 'float'}

In [7]:
env.action_space

MultiBinary(13)

In [8]:
# Run simple random policy
observation, info = env.reset()

print("Starting episode...")
print(f"Observation keys: {observation.keys()}")
print(f"Frame shape: {observation['frame'].shape}")
print(f"Info: {info}")
print("-" * 60)

for i in range(100):
    # Multi-binary action: each element toggles a key
    # For simple demo, just sample random actions
    action = env.action_space.sample()

    observation, reward, terminated, truncated, info = env.step(action)

    # Access normalized HP from info
    player_hp = info.get("player_hp_normalized", 0) * 100
    boss_hp = info.get("boss_hp_normalized", 100) * 100

    print(
        f"Step {i:3d} | Player HP: {player_hp:5.1f}% | Boss HP: {boss_hp:5.1f}% | Reward: {reward:+6.3f}"
    )

    # Small delay to make output readable
    time.sleep(0.2)

    if terminated or truncated:
        print("\n" + "=" * 60)
        if info.get("boss_hp_normalized", 1.0) <= 0:
            print("🎉 BOSS DEFEATED!")
        else:
            print("💀 YOU DIED")
        print("=" * 60 + "\n")

        # Reset for next episode
        observation, info = env.reset()

env.close()
print("\nEnvironment closed.")

Starting episode...
Observation keys: dict_keys(['frame', 'HeroHp', 'HeroMaxHp', 'NpcHp', 'NpcMaxHp', 'HeroAnimId', 'NpcAnimId'])
Frame shape: (2160, 3840, 3)
Info: {'player_hp_normalized': 1.0, 'boss_hp_normalized': 1.0, 'player_animation': 10000000, 'boss_animation': 2002000}
------------------------------------------------------------
Step   0 | Player HP: 100.0% | Boss HP: 100.0% | Reward: +0.000
Step   1 | Player HP: 100.0% | Boss HP: 100.0% | Reward: +0.000
Step   2 | Player HP: 100.0% | Boss HP: 100.0% | Reward: +0.000
Step   3 | Player HP: 100.0% | Boss HP: 100.0% | Reward: +0.000
Step   4 | Player HP: 100.0% | Boss HP: 100.0% | Reward: +0.000
Step   5 | Player HP:  96.8% | Boss HP: 100.0% | Reward: +0.000
Step   6 | Player HP:  96.8% | Boss HP: 100.0% | Reward: +0.000
Step   7 | Player HP:  96.8% | Boss HP: 100.0% | Reward: +0.000
Step   8 | Player HP:  96.8% | Boss HP: 100.0% | Reward: +0.000
Step   9 | Player HP:  96.8% | Boss HP: 100.0% | Reward: +0.000
Step  10 | Player HP

## Using Wrappers

EldenGym provides wrappers for preprocessing observations:


In [6]:
# Create environment with preprocessing wrappers
# env = eldengym.make(
#     "Margit-v0",
#     memory_attributes=["HeroHp", "HeroMaxHp", "NpcHp", "NpcMaxHp"],
#     frame_format="jpeg",
#     frame_quality=85,
# )

# # Apply wrappers
# env = eldengym.DictResizeFrame(env, width=84, height=84)  # Resize to 84x84
# env = eldengym.DictGrayscaleFrame(env)  # Convert to grayscale
# env = eldengym.DictFrameStack(env, num_stack=4)  # Stack 4 frames
# env = eldengym.NormalizeMemoryAttributes(env)  # Normalize memory values [0,1]

# print(f"Wrapped observation space: {env.observation_space}")

# # Test wrapped environment
# obs, info = env.reset()
# print(f"\nWrapped observation keys: {obs.keys()}")
# print(
#     f"Wrapped frame shape: {obs['frame'].shape}"
# )  # Should be (84, 84, 4) for grayscale 4-stack
# print(f"Normalized HeroHp: {obs['HeroHp']:.3f}")  # Should be in [0, 1]

# env.close()